In [1]:
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

def filter_words(story):
    story = story.split(' ')
    for i in story:
        if(i in stop_words):
            story.remove(i)
    return story

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\karth\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [22]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer

df_key = pd.read_csv('/Users/karth/OneDrive/Desktop/OELP/test/keywords.csv',delimiter='\t')
df  = pd.read_csv('/Users/karth/OneDrive/Desktop/OELP/test/features.csv',delimiter='\t')

s = set()                           # set of all unique keywords
for i in df_key['k_h']:
    x = eval(i)
    for j in x.keys():
        s.add(j)
print('unique keywords : ')
print(s,'\n')                               
M = []                             # M is the matrix where each row represents each movie and column represents each unique keyword
for i in df_key['k_h']:
    x = eval(i)
    l = []
    for j in s:
        if(j in x.keys()):
            l.append(x[j])
        else:
            l.append(0)
    M.append(l)
mat = np.array(M)
#print(mat)
sim_mat = cosine_similarity(mat)          # first similarity matrix which takes only keywords and its helpful score as features
print('similarity matrix for keywords as feaures : ')
print(sim_mat)

unique keywords : 
{'milking a cow', 'party', 'weightlifting', 'glow stick', 'shield wall', 'blood pool', 'bulletproof vest', 'laser', 'collaborationist', 'chimney', 'laser cutter', 'destroying evidence', 'fashion student', 'cancer treatment', 'voice over conversation', 'substance abuse', 'city in ruins', 'twin sisters played by same actress', 'medical prognosis', 'virginia', 'bestselling novel', 'playing a wooden flute', 'bad weather', '1190s', 'military dress uniform', 'volleyball ball', 'therizinosaurus', 'hit with a rock', 'dna test', 'one actress for twin sisters', 'stage actor', 'masked hero', 'jungle expedition', 'magic ritual', 'graphic violence', 'dystopian future', 'pta meeting', 'one room movie', 'newark new jersey', 'household servant', 'killing a snake', 'navy captain', 'gay cruising', 'cia officer', 'guy friend', 'race riot', 'suicide-by-cop', 'rapist', 'trapped for all eternity', 'reliving same event', 'drug smuggling', 'adrift', 'bowel', 'arizonan stereotype', 'cosmetic

In [23]:

df['avg_rating'] = df['avg_rating'].astype(str)
features = ['director','cast','avg_rating','writer','title']    # these are the features for second similarity matrix

def combine(df_row):                            # combining all the unique features in single line
    x = ''
    for i in features[:]:
        x+=df_row[i]
        x+=','
    story_arr = filter_words(df_row['storyline'])
    for i in story_arr:
        x+=i
        x+=','
    return x

cv = CountVectorizer()                                     # this counts frequency of each unique feature in each movie
df['final'] = df.apply(combine,axis=1)
count_matrix = cv.fit_transform(df['final'])
cos_sim = cosine_similarity(count_matrix)
print('unique feature words : ',cv.get_feature_names_out())
print('\n')
print('count matrix : ')
print(count_matrix.toarray())
print('\n')
print('similarity matrix : ')
print(cos_sim)


unique feature words :  ['000' '070' '10' ... 'zylik' 'zyrianova' 'zysermann']


count matrix : 
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


similarity matrix : 
[[1.         0.09392219 0.066541   ... 0.03902463 0.05975054 0.02773501]
 [0.09392219 1.         0.19651545 ... 0.14726566 0.17071922 0.09768499]
 [0.066541   0.19651545 1.         ... 0.06895064 0.11946104 0.05416186]
 ...
 [0.03902463 0.14726566 0.06895064 ... 1.         0.09100935 0.01159659]
 [0.05975054 0.17071922 0.11946104 ... 0.09100935 1.         0.03551104]
 [0.02773501 0.09768499 0.05416186 ... 0.01159659 0.03551104 1.        ]]


In [24]:
final = []                                                 # This is the final matrix which is combination of both the above matrices
for i in range(len(sim_mat)):
    row = []                                               # both rows and columns represents corresponding movies and the value in that entry will be final similarity score of those 2 movies
    for j in range(len(sim_mat[0])):
        b = ((0.6*sim_mat[i][j])+(0.4*cos_sim[i][j]))    # we are considering more probability (0.6) to first matrix because giving more priority to keywords and their helpful score will give more accurate recommendation
        row.append(b)                      # final similarity score = [(0.6)*(similarity score from keywords alone) + (0.4)*(similarity score from remaining features)] 
    final.append(row)                 # values 0.6 and 0.4 were decided by multiple trails and these values gave more accurate recommendation

titles = list(df['title'])                      # now from index of final matrix we will get titlee and story
story = list(df['storyline'])
t_i = {}
i_t = {}
for i in range(len(titles)):
    t_i[titles[i]] = i
    i_t[i] = titles[i]

def find(word):
    print('Movie given as input is: ',word,'\n')
    ind = t_i[word]
    inds = []
    m = list(final[ind]).copy()
    for i in range(11):
        x = max(m)                             # we are recommending top 10 highest similarity scores
        #print(x)
        j = m.index(x)
        #print(m.index(x))
        m[j] = -2
        inds.append(j)
    for i in inds[1:]:
        print(i_t[i],'  ',story[i])

find('The Boss Baby')


Movie given as input is:  The Boss Baby 

The Boss Baby: Family Business    The Templeton brothers have become adults and drifted away from each other, but a new boss baby with a cutting-edge approach is about to bring them together again - and inspire a new family business.
Knight of Cups    A writer indulging in all that Los Angeles and Las Vegas have to offer, undertakes a search for love and self via a series of adventures with six different women.
Creed II    Under the tutelage of Rocky Balboa, newly crowned heavyweight champion Adonis Creed faces off against Viktor Drago, the son of Ivan Drago.
The Son    Peter has his busy life with new partner Beth and their baby thrown into disarray when his ex-wife Kate turns up with their teenage son, Nicholas.
Armageddon Time    A deeply personal coming-of-age story about the strength of family and the generational pursuit of the American Dream.
Good Time    After a botched bank robbery lands his younger brother in prison, Connie Nikas emba